In [ ]:
import cv2 #OpenCV 영상처리

#classifier
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

gender_net = cv2.dnn.readNetFromCaffe('deploy_gender.prototxt','gender_net.caffemodel')

gender_list = ['Male', 'Female']
#video caputure setting
capture = cv2.VideoCapture(0) # initialize, # is camera number

count = 0 # # of caputre face images
#영상 처리 및 출력
while True: 
    ret, frame = capture.read() #카메라 상태 및 프레임
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #흑백으로
    faces = faceCascade.detectMultiScale(
        gray,#검출하고자 하는 원본이미지
        scaleFactor = 1.1, #검색 윈도우 확대 비율, 1보다 커야 한다
        minNeighbors = 5, #얼굴 사이 최소 간격(픽셀)
        minSize=(20,20) #얼굴 최소 크기. 이것보다 작으면 무시
    )
    print(count)

    #얼굴에 대해 rectangle 출력
    for (x,y,w,h) in faces:
        # 성별
        face = frame[int(y):int(y+h),int(x):int(x+h)].copy()
        blob = cv2.dnn.blobFromImage(face, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = gender_preds.argmax()

        info = gender_list[gender]        
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        count += 1
        cv2.imwrite("./gender/User."+str(count)+"."+str(info)+".jpg",frame[y:y+h, x:x+w])
    cv2.imshow('image',frame)

    #종료 조건
    if cv2.waitKey(1) == ord('q') : break # q키 입력이 있을 때 반복문 종료
    elif count >= 100 : break #100 face sample

print("\n [INFO] Exiting Program and cleanup stuff")

capture.release() #메모리 해제
cv2.destroyAllWindows()#모든 윈도우 창 닫기